In [1]:
%pip install datasets huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [14]:
import re

from datasets import load_dataset
from huggingface_hub import InferenceClient

In [78]:
dataset = load_dataset("HuggingFaceH4/self-instruct-seed", split='train').take(8)
dataset

Dataset({
    features: ['id', 'name', 'instruction', 'instances', 'is_classification'],
    num_rows: 8
})

In [16]:
def encode_prompt(prompt_instructions, classification=False):
    """Encode multiple prompt instructions into a single string."""
    if classification:
        prompt = "Come up with a series of classification tasks. Try to specify the possible output labels when possible.\n"
    else:
        prompt = "Come up with a series of tasks:\n"
    for idx, instruction in enumerate(prompt_instructions):
        instruction = re.sub(r"\s+", " ", instruction).strip().rstrip(":")
        prompt += f"{idx+1}. {instruction}\n"
    prompt += f"{len(prompt_instructions) + 1}."
    return prompt

In [18]:
prompt = encode_prompt(dataset['instruction'])
print(prompt)

Come up with a series of tasks:
1. Is there anything I can eat for a breakfast that doesn't include eggs, yet includes protein, and has roughly 700-1000 calories?
2. What is the relation between the given pairs?
3. Generate a one-sentence description for each of the following people.
4. Describe a situation in which the given stereotype can harm you.
5. Generate an appropriate subjective title for the following email
6. How do you answer this question in a job interview?
7. Brainstorm a list of possible New Year's resolutions.
8. Explain the following idiom to me, and try to give me some examples.
9.


In [75]:
client = InferenceClient('bigscience/bloom')

In [76]:
resp = client.text_generation(
    prompt, return_full_text=False, temperature=0.7, max_new_tokens=512
)
resp

' Think of a time when you were incredibly confident, and explain why.\n10. What is the difference between a real and normal friend?'

In [77]:
resp.split("\n10.")[0]

' Think of a time when you were incredibly confident, and explain why.'